In [ ]:
%%capture

!pip install gdown gpxpy

In [ ]:
%%capture

import gdown
DRIVE_FOLDER = 'https://drive.google.com/drive/folders/1SFjNDTuglgnWmVSY-I47xiekc-iqMCG_?usp=sharing'
gdown.download_folder(DRIVE_FOLDER, quiet=True)

In [ ]:
import gpxpy
import pandas as pd

def parse_gpx(filepath):
  gpx_file = open(filepath, 'r')
  gpx = gpxpy.parse(gpx_file)

  data = []

  for track in gpx.tracks:
    for segment in track.segments:
      for point in segment.points:
        time, lat, lng = point.time, point.latitude, point.longitude,
        data.append({
          'time': time,
          'lat': lat,
          'lon': lng,
        })

  df = pd.DataFrame(data)
  df['time'] = pd.to_datetime(df['time'])

  return df

osmtracker_df = parse_gpx('/content/multipath/osm-upload7163940511671199185.gpx')
osmtracker_df.head()

In [ ]:
constellation_names = {
    0: 'Unknown',
    1: 'GPS',
    3: 'Glonass',
    4: 'QZSS',
    5: 'Beidou',
    6: 'Galileo',
}

gnss_logger_df = pd.read_csv('/content/multipath/raw.csv')

gnss_logger_df['time'] = pd.to_datetime(gnss_logger_df['utcTimeMillis'], unit='ms', utc=True)
gnss_logger_df['ConstellationName'] = gnss_logger_df['ConstellationType'].map(constellation_names).astype(str)

gnss_logger_df.head()

In [ ]:
gnss_logger_df['ConstellationName'].value_counts()

In [ ]:
df = pd.merge(osmtracker_df, gnss_logger_df, on='time', how='inner')
df.head()

In [ ]:
import folium

m = folium.Map(
    location=(
      df['lat'].mean(),
      df['lon'].mean(),
    ),
    zoom_start=18,
)

marker_colors = {0: 'black', 1: 'red', 2: 'green'}

sampled_df = df[df['ConstellationName'] == 'QZSS']
for index, row in sampled_df.iterrows():
    folium.Marker(
        [row['lat'], row['lon']],
        popup=f"Svid: {row['Svid']}, Multipath: {row['MultipathIndicator']}",
        icon=folium.Icon(color=marker_colors.get(row['MultipathIndicator']))
    ).add_to(m)

m

In [ ]:
import folium

m = folium.Map(
    location=(
      df['lat'].mean(),
      df['lon'].mean(),
    ),
    zoom_start=18,
)

marker_colors = {0: 'black', 1: 'red', 2: 'green'}

sampled_df = df[df['ConstellationName'] == 'Galileo']
for index, row in sampled_df.iterrows():
    folium.Marker(
        [row['lat'], row['lon']],
        popup=f"Svid: {row['Svid']}, Multipath: {row['MultipathIndicator']}",
        icon=folium.Icon(color=marker_colors.get(row['MultipathIndicator']))
    ).add_to(m)

m

In [ ]:
import folium

m = folium.Map(
    location=(
      df['lat'].mean(),
      df['lon'].mean(),
    ),
    zoom_start=18,
)

marker_colors = {0: 'black', 1: 'red', 2: 'green'}

sampled_df = df[df['ConstellationName'] == 'Glonass']
for index, row in sampled_df.iterrows():
    folium.Marker(
        [row['lat'], row['lon']],
        popup=f"Svid: {row['Svid']}, Multipath: {row['MultipathIndicator']}",
        icon=folium.Icon(color=marker_colors.get(row['MultipathIndicator']))
    ).add_to(m)

m